# Introduction to the Monte Carlo method

----

Start by defining the [Gibbs (or Boltzmann) distribution](https://en.wikipedia.org/wiki/Boltzmann_distribution):
$$P(\alpha) = e^{-E(\alpha)/kT}$$
this expression, defines the probability of observing a particular configuration of spins, $\alpha$. 
As you can see, the probability of $\alpha$ decays exponentially with increasing energy of $\alpha$, $E(\alpha)$,
where $k$ is the Boltzmann constant, $k = 1.38064852 \times 10^{-23} J/K$
and $T$ is the temperature in Kelvin. 

## What defines the energy of a configuration of spins? 
Given a configuration of spins (e.g., $\uparrow\downarrow\downarrow\uparrow\downarrow$) we can define the energy using what is referred to as an Ising Hamiltonian:
$$ \hat{H}' = \frac{\hat{H}}{k} = -\frac{J}{k}\sum_{<ij>} s_is_j,$$
where, $s_i=1$ if the $i^{th}$ spin is `up` and $s_i=-1$ if it is `down`, and the brackets $<ij>$ indicate a sum over spins that are connected,
and $J$ is a constant that determines the energy scale. 
The energy here has been divided by the Boltzmann constant to yield units of temperature. 
Let's consider the following case, which has the sites connected in a single 1D line:
$$\alpha = \uparrow-\downarrow-\downarrow-\uparrow-\downarrow.$$ 
What is the energy of such a configuration?
$$ E(\alpha)' = J/k(-1 + 1 - 1 - 1) = \frac{E(\alpha)}{k} = -2J/k$$

## P1: Write a class that defines a spin configuration

In [1]:
class spin_config:
    def __init__(self):
        self.config = []
        self.N = 0
    def n_sites(self):
        return len(self.config)
    def initialize(self,list):
        for i in list:
            self.config.append(i)
    def reset(self):
        self.config.clear()
    def configuration(self):
        return self.config
        

## P2: Write a class that defines the 1D hamiltonian, containing a function that computes the energy of a configuration

In [2]:
class Hamiltonian:
    def __init__(self,J = -2,mu=1.1,k=1):
        self.energy = 0
        self.state = []
        self.j = J
        self.mu = mu
        self.k = k
    def initialize(self,list):
        for i in list:
            self.state.append(i)
    def reset(self):
        self.state.clear()
        self.energy = 0
    def spin_energy(self):
        for i in range(len(self.state)-1):
            self.energy+=(self.state[i]*self.state[i+1])
        self.energy+=self.state[0]*self.state[-1]
        self.energy*= -self.j/self.k
    
        for j in self.state:
            self.energy+=(self.mu*j)
            
        return self.energy        

## Q3: What is the energy for (++-+---+--+)?

In [3]:
lattice_energy = Hamiltonian()
list = [1,1,-1,1,-1,-1,-1,1,-1,-1,1]
lattice_energy.initialize(list)
lattice_energy.spin_energy()

-3.1

## Properties
For any fixed state, $\alpha$, the `magnetization` ($M$) is proportional to the _excess_ number of spins pointing up or down while the energy is given by the
Hamiltonian:
$$M(\alpha) = N_{\text{up}}(\alpha) - N_{\text{down}}(\alpha).$$
As a dynamical, fluctuating system, each time you measure the magnetization, the system might be in a different state ($\alpha$) and so you'll get a different number!
However, we already know what the probability of measuring any particular $\alpha$ is, so in order to compute the average magnetization, $\left<M\right>$, we just need to multiply the magnetization of each possible configuration times the probability of it being measured, and then add them all up!
$$ \left<M\right> = \sum_\alpha M(\alpha)P(\alpha).$$
In fact, any average value can be obtained by adding up the value of an individual configuration multiplied by it's probability:
$$ \left<E\right> = \sum_\alpha E(\alpha)P(\alpha).$$

This means that to obtain any average value (also known as an `expectation value`) computationally, we must compute the both the value and probability of all possible configurations. This becomes extremely expensive as the number of spins ($N$) increases. 

## P3: Write a function that computes the magnetization of a spin configuration

In [4]:
def magnetization(spin_config):
    magnet = 0
    for i in spin_config:
        if i==1:
            magnet+=1
        else:
            magnet-=1

    return magnet

## Average Energy, Average Magnetization, Heat Capacity, and Magnetic Susceptibility

In [1]:
import numpy as np
import matplotlib.pyplot as plt

class spin_config_1D:
    def __init__(self,boltz=1,pref=-2,mu=1.1,temp=373):
        self.energies = []
        self.magnetizations = []
        self.probabilities = []
        self.boltzmann = []
        self.states = []
        self.k = boltz
        self.j = pref
        self.mu = mu
        self.T = temp
        self.avg_eng = 0
        self.avg_mag = 0
        self.heat_capacity = 0
        self.mag_sus = 0
    def reset(self):
        self.energies = []
        self.magnetizations = []
        self.probabilities = []
        self.boltzmann = []
        self.states = []
        self.k = 1
        self.j = -2
        self.mu = 1.1
        self.T = 323
        self.avg_eng = 0
        self.avg_mag = 0
        self.heat_capacity = 0
        self.mag_sus = 0
    def clean(self):
        self.energies = []
        self.magnetizations = []
        self.probabilities = []
        self.boltzmann = []
        self.states = []
        self.avg_eng = 0
        self.avg_mag = 0
        self.heat_capacity = 0
        self.mag_sus = 0
    def temp(self,temp):
        self.T = temp
    def J(self,pref):
        self.j = pref
    def boltz(self,k):
        self.k = boltz
    def mu(self,m):
        self.mu = m
    def report(self):
        print("States:",self.states)
        print()
        print("Energies:",self.energies)
        print()
        print("Magnetizations:",self.magnetizations)
        print()
        print("Probabilities:",self.probabilities)
        print()
        print("Average Energy:",self.avg_eng)
        print()
        print("Average Magnetization:",self.avg_mag)
        print()
        print("Heat Capacity:",self.heat_capacity)
        print()
        print("Magnetic Susceptibility:",self.mag_sus)
        print()
        print("Constants-")
        print("\tBoltzmann Constant is: ",self.k)
        print("\tJ is: ",self.j)
        print("\tmu is: ",self.mu)
        print("\tTemperature is: ",self.T)
        
    def constants(self,boltz,pref,mu,temp):
        self.k = boltz
        self.j = pref
        self.mu = mu
        self.T = temp
    def generate_state(self,n=8):
        self.clean()
        for i in range(2**n):
            binary = bin(i)
            state = [char for char in binary]
            state.remove('0')
            state.remove('b')
            for j in range(len(state)):
                state[j] = int(state[j])
                if state[j]==0:
                    state[j]=-1
            while len(state)<n:
                state.insert(0,-1)
            self.states.append(state)
        #Generates the states
        
        for j in self.states:
            energy = 0
            for i in range(len(j)-1):
                energy+=(j[i]*j[i+1])
            energy+=j[-1]*j[0]
            energy *= -self.j/self.k
            for k in j:
                energy+=(self.mu*k)
            self.energies.append(energy)
        #Generates the energies of each state    
        
        for j in self.states:
            magnet = 0
            for i in j:
                magnet+=i
            self.magnetizations.append(magnet)
        #Generates the magnetizations of each state    
        
        for i in self.energies:
            self.boltzmann.append(np.exp(-i/(self.k*self.T)))
        normalization_factor = sum(self.boltzmann)
        
        #Generates the probability graph, unnormalized as a list
        
        for j in range(len(self.boltzmann)):
            self.probabilities.append(self.boltzmann[j]/normalization_factor)
            
        #Generates a new list, normalized this time
        
        for i in range(len(self.energies)):
            self.avg_eng+=self.energies[i]*self.boltzmann[i]
        self.avg_eng/=normalization_factor
        
        #Computes average energy
        for i in range(len(self.magnetizations)):
            self.avg_mag+=self.magnetizations[i]*self.boltzmann[i]
        self.avg_mag/=normalization_factor
        
        #Computes average magnetization
        
        copy_energy=[]
        E = 0        
        for i in self.energies:
            copy_energy.append(i**2)
        for i in range(len(copy_energy)):
            E+=copy_energy[i]*self.boltzmann[i]
        E/=normalization_factor
        self.heat_capacity = (E - np.power(self.avg_eng,2))/(self.k*self.T*self.T)
        
        #Computes heat capacity
        
        copy_mag=[]
        M = 0        
        for i in range(len(self.magnetizations)):
            copy_mag.append(self.magnetizations[i]**2)
        for i in range(len(copy_mag)):
            M+=copy_mag[i]*self.boltzmann[i]
        M/=normalization_factor
        self.mag_sus = (M - np.power(self.avg_mag,2))/(self.k*self.T)
        
        #Computes magnetic suscpetibility
        
    def generate_plot(self,tmin=1,tmax=10,step=0.1,num_states=8):
        t_current = self.T
        temps = []
        average_energy = []
        average_magnetization=[]
        heat_cap=[]
        mag_sus=[]
        for i in range(int(tmin),int(tmax/step)):
            temps.append(i*step)
            self.temp(i*step)
            self.generate_state(num_states)
            average_energy.append(self.avg_eng)
            average_magnetization.append(self.avg_mag)
            heat_cap.append(self.heat_capacity)
            mag_sus.append(self.mag_sus)
        plt.plot(temps,average_energy,'r-',temps,average_magnetization,'b-',temps,heat_cap,'g-',temps,mag_sus,'y-')
        plt.legend(["Average Energy", "Average Magnetization", "Heat Capacity", "Magnetic Susceptibility"],loc='best')
        plt.xlabel("Temperature (K)")
        self.T = t_current

## Q2: How many configurations are possible for:

(a) N=10?

$2^{10}$

(b) N=100?

$2^{100}$

(c) N=1000?

$2^{1000}$